# Face to Face Conversation System using Deep Neural Networks

The project involves the following steps taken by Rahul Sharma with reference from the research paper.<br>

1. Create a dummy dataset(since collection of original dataset will take a lot of time)<br>
2. Create an encoder decoder architecture for the speaking and listening model
3. Implement the conversational model
4. Test the speaking and listening models
5. Take the output of the speaking and listening models
6. Implement Pix2Pix HD GAN to get the final output

In the middle there will be certain tests to perform which will come as the project proceeds.

## 1. Creation of a dummy dataset

The dummy dataset needs the Facial Action Units + Face Pose of the listener and speaker. These are 20 dimensional vectors which are created arbitrarily from the 